In [10]:

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import random

from selenium.webdriver.common.action_chains import ActionChains


In [2]:
from pypasser import reCaptchaV3
reCaptcha_response = reCaptchaV3('https://www.google.com/recaptcha/api2/anchor?ar=1&k=6LeuwcIUAAAAAOxIwk4Ji5ZrvseZel4_ufSQ7Njg&co=aHR0cHM6Ly9wcmVwYWlkLnVzY2VsbHVsYXIuY29tOjQ0Mw..&hl=en&type=image&v=3sU2vDRVDmUU2E0Ro4VadvPr&theme=light&size=invisible&badge=bottomright&cb=8fev9ltzdx1')

C:\Users\ericluo\AppData\Roaming\Python\Python39\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
reCaptcha_response

'03AFcWeA5nOBy6A3U4IXCGVp69r3bnYOHY-s_F88uPUdMVMqk58YahZQsQHI__dlRM0BmkzHPAOZk2yRDp5n7CcUVvC6HSUz7XpfZSHnNqs8pUtqoxF8cfqduLIOn1bbT8K7hNUNvfeiwn2FyFJz1uxIv49Chmv9dvLi3g4iqdHna37FjsdogzUQF_k9k-CoimU8VVxGfQpryKGDKVKWoXKLUzF-O9iDvttmE_lFrO4Pt_Q4ipLRb1i5jF-jFL6S0lMKnpPpPVTFznKuDe09llMSLcrK-hDFIJrPN-MAGBOxumtK2JVnuxBdia_uQ1MZTt3szywMZpU_1hDutzsSBU5EWyvtsahZPjwcNTypg6gCug7HfWXYHAv3PA4Iof3sewOjjO7EEzkL42d8tGtxar6oSwepKIXIIbOdVDEBrkjVr_4VBaraV_8_q-_cHRQsvLd05h69b9Oid2sX-lVTgoQoXmXxr1f8s45GHAZOk78BWMdOBy-zC5T3UuTEITwACEqkxun8hDbdiamhVM5oS2oKeCakd2fJsExjtt9_Vpa86f8CDFia7ZdAKiLjo6z3QRpkmFwkMzrsKgA9Z7lzg7GjR7-tHnxYoWL729oclFuBmm7pdiZKF-frLDXuq_QFqGoSiLbLQPiL9fG6DV8HFMjl2KBskn_HPOLVQ35wr1pCgKETA93OrIyd6iIDlM4BEFMvIhE2aJcYH7mp-CuAT9aXzTPF1GJcQ_guHogA_j48Mn7QiENbFF0c_wOGJr9OKojy3_nMJeEaU7efOV5KR0obnh1R18SgAKxsVqxyyaJstzPaRgPQreH31e_Ur75_0m7h3I54Ills2SvY51Hr6HFGAD2MZIokZ7hFrqp2icQ-37-oT0OXQ9N65MT79DqyEuo8Gb3YNxHOYRv7upgVwWGo6JKtmSjKLYtewpOZ22ezAlge_J4hbrOXARuuC41Tmx0LofpIaAS5i0BZOKbU-4jmNwMJyMV2E0ShFN8

In [11]:
def perform_random_click(driver, element):
    """Perform a random click near the given element."""
    action = ActionChains(driver)
    # Get the size of the element
    size = element.size
    # Calculate random position around the element (within 10 pixels)
    x_offset = random.randint(-10, size['width'] + 10)
    y_offset = random.randint(-10, size['height'] + 10)
    # Move to the element with offset and click
    action.move_to_element_with_offset(element, x_offset, y_offset).click().perform()

# Function to get error code based on the error message
def get_error_code(error_message):
    if 'billed customer' in error_message:
        return 1
    elif 'Phone number not valid' in error_message:
        return 3
    elif 'no longer in service' in error_message:
        return 2
    elif 'Please verify you are a human before continuing' in error_message:
        return 4
    elif 'No errors, it is a prepaid number' in error_message:
        return 0
    else:
        return -1  # Unknown error message

In [12]:
# Read phone numbers from file and remove country code
with open('test_panel_usus.txt', 'r') as file:
    phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file][:]
phone_numbers[0]

'2408071029'

In [29]:
options = webdriver.ChromeOptions()
options.headless = True

results = []
consecutive_error_count = 0
current_index = 0  # Initialize the index to keep track of the current phone number

def init_driver():
    """Initialize the Chrome WebDriver."""
    driver = webdriver.Chrome(options=options)
    driver.get('https://prepaid.uscellular.com/refill/dashboard')
    WebDriverWait(driver, 1).until(
        EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    ).click()
    return driver


# def init_driver():
#     """Initialize the Chrome WebDriver with incognito mode."""
#     options = webdriver.ChromeOptions()
#     options.headless = True
#     options.add_argument("--incognito")  # Add this line for incognito mode
#     # ... [Any other options you wish to include]

#     driver = webdriver.Chrome(options=options)
#     driver.get('https://prepaid.uscellular.com/refill/dashboard')
#     WebDriverWait(driver, 1).until(
#         EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
#     ).click()
#     return driver

# Initialize the WebDriver for the first time
driver = init_driver()

while current_index < len(phone_numbers):
    print(current_index)  
    try:
        phone_number = phone_numbers[current_index]
        print(f"Processing phone number {phone_number} at index {current_index}")

        # phone number input field
        phone_input_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "phone"))
        )
        phone_input_field.click()  # Focus on the input field

        # Clear the input field 
        phone_input_field.send_keys(Keys.CONTROL + "a")
        phone_input_field.send_keys(Keys.DELETE)
        
        # Clear the input field and input phone number
        # phone_input_field.clear()
        for i in range(len(phone_number)):
            char = phone_number[i]
            phone_input_field.send_keys(char)
            time.sleep(random.uniform(0.3, 1))  # Small delay between each character


        # phone number input field

            # Randomly decide whether to delete the last few digits and re-enter them
            if random.random() < 0.3:  # 30% chance to delete and re-enter
                # Randomly choose how many characters to delete (1 to 2 characters in this case)
                chars_to_delete = random.randint(1, 2)
                # Ensure we do not delete more characters than we have typed
                chars_to_delete = min(chars_to_delete, i + 1)

                # Delete the characters
                for _ in range(chars_to_delete):
                    phone_input_field.send_keys(Keys.BACKSPACE)
                    time.sleep(random.uniform(0.1, 1))
                    # # Simulate random clicking while typing
                    # if random.random() < 0.1:  # 10% chance to click randomly on the page
                    #     perform_random_click(driver, phone_input_field)                    

                # Re-enter the deleted characters
                for j in range(i - chars_to_delete + 1, i + 1):
                    phone_input_field.send_keys(phone_number[j])
                    time.sleep(random.uniform(0.3, 1))
                    # # Simulate random clicking while typing
                    # if random.random() < 0.1:  # 10% chance to click randomly on the page
                    #     perform_random_click(driver, phone_input_field)
        # Simulate random clicking while typing
        if random.random() < 0.5:  # 10% chance to click randomly on the page
            perform_random_click(driver, phone_input_field)
        time.sleep(random.uniform(2, 4))  # Less delay after finishing typing

        # time.sleep(random.uniform(2, 20))

        # # default error message
        # error_message = "No errors, it is a prepaid number"
        # error_code = get_error_code(error_message)  # Get error code
        # print(error_code)

        # Check for an error message, wait up to 5 seconds
        try:
            error_message_xpath = "/html/body/div[1]/main/div[1]/div/div[2]/div/div[2]/div[1]/div/div[2]/div/div/div/div/form/div[1]/div[1]/div[2]"
            error_message_element = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.XPATH, error_message_xpath))
            )
            if error_message_element.text.strip():  # If there's text in the error element
                error_message = error_message_element.text
                error_code = get_error_code(error_message)  # Update error code
                print(error_message,error_code)
                results.append((phone_number, error_code))  # Append to results
        except TimeoutException:
            # current_index -= 1
            print('current_index ',current_index)  
            # If no error message appeared within 5 seconds, assume no error
            continue

        # Reset consecutive error count if no error occurred
        if error_code == 0:
            consecutive_error_count = 0
            time.sleep(random.uniform(30, 35))
            current_index += 1  # Increment the index after processing
        if error_code == 4:
            driver.refresh()
            continue
        else:
            time.sleep(random.uniform(30, 35))
            current_index += 1  # Increment the index after processing
            consecutive_error_count += 1
            if consecutive_error_count == 2:  # If three consecutive errors, reset
                driver.refresh()
                time.sleep(random.uniform(30, 35))
                # driver.quit()
                # time.sleep(random.uniform(2, 30))
                # driver = init_driver()
                consecutive_error_count = 0 

    except Exception as e:
        print(f"An error occurred: {e}. Retrying the current phone number. Current index: {current_index}")
        time.sleep(random.uniform(30, 35))
        driver.refresh()
        # driver.quit()
        # driver = init_driver()  # Re-initialize the WebDriver
        # print(current_index)   

# Clean up
driver.quit()


0
Processing phone number 2408071029 at index 0
current_index  0
0
Processing phone number 2408071029 at index 0
Phone number not valid 3
1
Processing phone number 3368268177 at index 1
Phone number not valid 3
2
Processing phone number 2623536856 at index 2
It looks like you’re a UScellular billed customer. To utilize Quick Pay for our billed customers Click Here 1
3
Processing phone number 8436556259 at index 3
current_index  3
3
Processing phone number 8436556259 at index 3
An error occurred: Message: element click intercepted: Element <input aria-invalid="false" name="phone" placeholder=" " type="text" inputmode="numeric" aria-label="UScellular Prepaid Number" class="MuiInputBase-input MuiOutlinedInput-input MuiInputBase-inputAdornedEnd MuiOutlinedInput-inputAdornedEnd" value="(843) 655-6259"> is not clickable at point (504, 25). Other element would receive the click: <iframe title="recaptcha challenge expires in two minutes" name="c-tu4k4hj7ivom" frameborder="0" scrolling="no" san

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=118.0.5993.120)
Stacktrace:
	GetHandleVerifier [0x00007FF67E158EF2+54786]
	(No symbol) [0x00007FF67E0C5612]
	(No symbol) [0x00007FF67DF7A64B]
	(No symbol) [0x00007FF67DF5A9F5]
	(No symbol) [0x00007FF67DFE0887]
	(No symbol) [0x00007FF67DFF43CF]
	(No symbol) [0x00007FF67DFDBC43]
	(No symbol) [0x00007FF67DFB0941]
	(No symbol) [0x00007FF67DFB1B84]
	GetHandleVerifier [0x00007FF67E4A7F52+3524194]
	GetHandleVerifier [0x00007FF67E4FD800+3874576]
	GetHandleVerifier [0x00007FF67E4F5D7F+3843215]
	GetHandleVerifier [0x00007FF67E1F5086+694166]
	(No symbol) [0x00007FF67E0D0A88]
	(No symbol) [0x00007FF67E0CCA94]
	(No symbol) [0x00007FF67E0CCBC2]
	(No symbol) [0x00007FF67E0BCC83]
	BaseThreadInitThunk [0x00007FFB0442257D+29]
	RtlUserThreadStart [0x00007FFB0670AA78+40]


In [13]:
# import asyncio
# from pyppeteer import launch
# from pyppeteer.errors import TimeoutError

# results = []
# consecutive_error_count = 0
# current_index = 0


# async def init_browser():
#     browser = await launch(headless=True, args=['--no-sandbox'])
#     page = await browser.newPage()
#     await page.goto('https://prepaid.uscellular.com/refill/dashboard')
#     # Wait for the selector and click on the reject all handler
#     await page.waitForSelector('#onetrust-reject-all-handler', options={'visible': True})
#     await page.click('#onetrust-reject-all-handler')
#     return page

# async def main(phone_numbers):
#     global current_index, consecutive_error_count
#     page = await init_browser()

#     while current_index < len(phone_numbers):
#         try:
#             phone_number = phone_numbers[current_index]
#             print(f'Processing phone number {phone_number} at index {current_index}')

#             # Wait for the phone input field and type the phone number
#             await page.waitForSelector('input[name="phone"]', options={'visible': True})
#             await page.click('input[name="phone"]')
#             await page.keyboard.press('Control+A')
#             await page.keyboard.press('Backspace')

#             for i, char in enumerate(phone_number):
#                 await page.keyboard.type(char)
#                 await asyncio.sleep(random.uniform(0.3, 1))

#                 # Randomly decide whether to delete the last few digits and re-enter them
#                 if random.random() < 0.3:  # 30% chance to delete and re-enter
#                     # Randomly choose how many characters to delete (1 to 2 characters in this case)
#                     chars_to_delete = random.randint(1, 2)
#                     # Ensure we do not delete more characters than we have typed
#                     chars_to_delete = min(chars_to_delete, i + 1)

#                     # Delete the characters
#                     for _ in range(chars_to_delete):
#                         await page.keyboard.press('Backspace')
#                         await asyncio.sleep(random.uniform(0.1, 1))

#                     # Re-enter the deleted characters
#                     for j in range(i - chars_to_delete + 1, i + 1):
#                         await page.keyboard.type(phone_number[j])
#                         await asyncio.sleep(random.uniform(0.3, 1))

#             await asyncio.sleep(random.uniform(2, 4))  # Less delay after finishing typing

#             # Check for an error message, wait up to 5 seconds
#             try:
#                 error_message_xpath = "/html/body/div[1]/main/div[1]/div/div[2]/div/div[2]/div[1]/div/div[2]/div/div/div/div/form/div[1]/div[1]/div[2]"
#                 error_message_element = await page.waitForXPath(error_message_xpath, {'visible': True, 'timeout': 5000})
#                 error_message = await page.evaluate('(element) => element.textContent', error_message_element)
#                 if error_message.strip():  # If there's text in the error element
#                     error_code = get_error_code(error_message)  # Update error code
#                     print(error_message, error_code)
#                     results.append((phone_number, error_code))
#             except TimeoutError:
#                 current_index -= 1
#                 print('current_index ', current_index)  

#             if error_code == 0:
#                 consecutive_error_count = 0
#                 current_index += 1
#             else:
#                 current_index += 1
#                 consecutive_error_count += 1
#                 if consecutive_error_count == 2:
#                     await page.browser.close()  # Corrected line here
#                     await asyncio.sleep(random.uniform(2, 30))
#                     page = await init_browser()
#                     consecutive_error_count = 0

#         except Exception as e:
#             print(f'An error occurred: {e}. Retrying the current phone number. Current index: {current_index}')
#             await page.browser.close()  # Corrected line here
#             page = await init_browser()

#     await page.browser.close()  # And also here




# loop = asyncio.get_event_loop()

# # If the loop is already running, we simply create a task.
# # If the loop is not running (like in standard Python script),
# # we run the loop until the coroutine is complete.
# if loop.is_running():
#     loop.create_task(main(phone_numbers))
# else:
#     loop.run_until_complete(main(phone_numbers))

Processing phone number 2408071029 at index 0
An error occurred: Unknown key: Control+A. Retrying the current phone number. Current index: 0
Processing phone number 2408071029 at index 0
An error occurred: Unknown key: Control+A. Retrying the current phone number. Current index: 0
Processing phone number 2408071029 at index 0
An error occurred: Unknown key: Control+A. Retrying the current phone number. Current index: 0
Processing phone number 2408071029 at index 0
An error occurred: Unknown key: Control+A. Retrying the current phone number. Current index: 0
Processing phone number 2408071029 at index 0
An error occurred: Unknown key: Control+A. Retrying the current phone number. Current index: 0
Processing phone number 2408071029 at index 0
An error occurred: Unknown key: Control+A. Retrying the current phone number. Current index: 0
Processing phone number 2408071029 at index 0
An error occurred: Unknown key: Control+A. Retrying the current phone number. Current index: 0
Processing ph

In [8]:
df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
df_results

,Phone Number,Error Code


In [9]:
# Convert the results to a DataFrame 
df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
print(df_results)

Empty DataFrame
Columns: [Phone Number, Error Code]
Index: []


In [8]:
import os

csv_file_path = 'uscs_query_result.csv'

df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
if os.path.exists(csv_file_path):
    df_existing = pd.read_csv(csv_file_path)
    
    last_index = df_existing.index[-1] if not df_existing.empty else -1

    df_results.index = range(last_index + 1, last_index + 1 + len(df_results))
    
    # Append the new results to the existing DataFrame
    df_final = df_existing.append(df_results)
    df_final = df_final.iloc[last_index + 1:,1:]
else:
    df_final = df_results
    df_final.index = range(len(df_final))

# Output to CSV, without overwriting existing data and with continuous indexing
df_final.to_csv(csv_file_path, index_label='Index', mode='w' if not os.path.exists(csv_file_path) else 'a', header=not os.path.exists(csv_file_path))

print(f"Results have been appended to {csv_file_path}")


Results have been appended to uscs_query_result.csv
